<a href="https://colab.research.google.com/github/abdallah388687/PySpark_Practical-Lab/blob/main/NLP_using_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![image.png](attachment:image.png)

##### **Good luck with taking your exam. Keep working and make your dreams all come true. Seeing the results of all of your hard work will make this struggle worth it. We’re all thinking of you.** 
<b><font color='blue'>AI-PRO Spark Team ITI</font></b>

# NLP Using PySpark

## Objective:
- The objective from this project is to create a <b>Spam filter using NaiveBayes classifier</b>.
- It is required to obtain <b>f1_scored > 0.9</b>.
- We'll use a dataset from UCI Repository. SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
- Data is also provided for you in the assignment (you do not have to download it).

## To perform this task follow the following guiding steps:

### Create a spark session and import the required libraries

In [ ]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

# install findspark using pip
!pip install -q findspark

!pip install pyspark

import findspark
findspark.init()

import pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#create object from SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

### Read the readme file to learn more about the data

### Read the data into a DataFrame

In [ ]:
df = spark.read.format("csv").option("header", "False").option("delimiter", "\t").load("SMSSpamCollection",inferSchema = True)

In [ ]:
df.show(2)

+---+--------------------+
|_c0|                 _c1|
+---+--------------------+
|ham|Go until jurong p...|
|ham|Ok lar... Joking ...|
+---+--------------------+
only showing top 2 rows



### Print the schema

In [ ]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



### Rename the first column to 'class' and second column to 'text'

In [ ]:
df = df.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
df.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
df.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)



### Show the first 10 rows from the dataframe
- Show once with truncate=True and once with truncate=False

In [ ]:
df.show(10,truncate = True)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
+-----+--------------------+
only showing top 10 rows



In [ ]:
df.show(10,truncate = False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                 |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                   |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075o

In [ ]:
df.select('class').groupby('class').count().show()

+-----+-----+
|class|count|
+-----+-----+
|  ham| 4827|
| spam|  747|
+-----+-----+



## Clean and Prepare the Data

### Create a new feature column contains the length of the text column

In [ ]:
from pyspark.sql.functions import *
df = df.withColumn("length", size(split(col("text"), r"")) - 1 )

### Show the new dataframe

In [ ]:
df.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



### Get the average text length for each class (give alias name to the average length column)

In [ ]:
df.groupBy('class').agg(avg('length').alias('Avg. Lenght')).show()

+-----+-----------------+
|class|      Avg. Lenght|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

### In this part you transform you raw text in to tf_idf model :
- For more information about TF-IDF check the following link: <b>(Not needed for the test)</b>
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

### Perform the following steps to obtain TF-IDF:
1. Import the required transformers/estimators for the subsequent steps.
2. Create a <b>Tokenizer</b> from the text column.
3. Create a <b>StopWordsRemover</b> to remove the <b>stop words</b> from the column obtained from the <b>Tokenizer</b>.
4. Create a <b>CountVectorizer</b> after removing the <b>stop words</b>.
5. Create the <b>TF-IDF</b> from the <b>CountVectorizer</b>.

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="Tokens")

In [ ]:
#Test cell
df_tokenized = tokenizer.transform(df)
df_tokenized.show(5)

+-----+--------------------+------+--------------------+
|class|                text|length|              Tokens|
+-----+--------------------+------+--------------------+
|  ham|Go until jurong p...|   111|[go, until, juron...|
|  ham|Ok lar... Joking ...|    29|[ok, lar..., joki...|
| spam|Free entry in 2 a...|   155|[free, entry, in,...|
|  ham|U dun say so earl...|    49|[u, dun, say, so,...|
|  ham|Nah I don't think...|    61|[nah, i, don't, t...|
+-----+--------------------+------+--------------------+
only showing top 5 rows



In [ ]:
Remover = StopWordsRemover(inputCol='Tokens', outputCol='Cleaned_Tokens')

In [ ]:
# Test cell
df_cleaned = Remover.transform(df_tokenized)
df_cleaned.show(5)

+-----+--------------------+------+--------------------+--------------------+
|class|                text|length|              Tokens|      Cleaned_Tokens|
+-----+--------------------+------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|[go, until, juron...|[go, jurong, poin...|
|  ham|Ok lar... Joking ...|    29|[ok, lar..., joki...|[ok, lar..., joki...|
| spam|Free entry in 2 a...|   155|[free, entry, in,...|[free, entry, 2, ...|
|  ham|U dun say so earl...|    49|[u, dun, say, so,...|[u, dun, say, ear...|
|  ham|Nah I don't think...|    61|[nah, i, don't, t...|[nah, think, goes...|
+-----+--------------------+------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
Count_Vectorizer = CountVectorizer(inputCol='Cleaned_Tokens', outputCol='CountVectorizerOutput')

In [ ]:
#TestCell
df_CountVectorized = Count_Vectorizer.fit(df_cleaned).transform(df_cleaned)
df_CountVectorized.show(5)

+-----+--------------------+------+--------------------+--------------------+---------------------+
|class|                text|length|              Tokens|      Cleaned_Tokens|CountVectorizerOutput|
+-----+--------------------+------+--------------------+--------------------+---------------------+
|  ham|Go until jurong p...|   111|[go, until, juron...|[go, jurong, poin...| (13423,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|[ok, lar..., joki...|[ok, lar..., joki...| (13423,[0,24,297,...|
| spam|Free entry in 2 a...|   155|[free, entry, in,...|[free, entry, 2, ...| (13423,[2,13,19,3...|
|  ham|U dun say so earl...|    49|[u, dun, say, so,...|[u, dun, say, ear...| (13423,[0,70,80,1...|
|  ham|Nah I don't think...|    61|[nah, i, don't, t...|[nah, think, goes...| (13423,[36,134,31...|
+-----+--------------------+------+--------------------+--------------------+---------------------+
only showing top 5 rows



In [ ]:
idf = IDF(inputCol="CountVectorizerOutput", outputCol="tf-idf_features")

In [ ]:
#Test Cell
idf_model = idf.fit(df_CountVectorized)
df_tfidf = idf_model.transform(df_CountVectorized)
df_tfidf.show(5)

+-----+--------------------+------+--------------------+--------------------+---------------------+--------------------+
|class|                text|length|              Tokens|      Cleaned_Tokens|CountVectorizerOutput|     tf-idf_features|
+-----+--------------------+------+--------------------+--------------------+---------------------+--------------------+
|  ham|Go until jurong p...|   111|[go, until, juron...|[go, jurong, poin...| (13423,[7,11,31,6...|(13423,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|[ok, lar..., joki...|[ok, lar..., joki...| (13423,[0,24,297,...|(13423,[0,24,297,...|
| spam|Free entry in 2 a...|   155|[free, entry, in,...|[free, entry, 2, ...| (13423,[2,13,19,3...|(13423,[2,13,19,3...|
|  ham|U dun say so earl...|    49|[u, dun, say, so,...|[u, dun, say, ear...| (13423,[0,70,80,1...|(13423,[0,70,80,1...|
|  ham|Nah I don't think...|    61|[nah, i, don't, t...|[nah, think, goes...| (13423,[36,134,31...|(13423,[36,134,31...|
+-----+--------------------+----

- Convert the <b>class column</b> to index using <b>StringIndexer</b>
- Create feature column from the <b>TF-IDF</b> and <b>lenght</b> columns.

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline

In [ ]:
stringIndexer = StringIndexer(inputCols=['class'], outputCols=['label'], handleInvalid='skip')
vecAssembler = VectorAssembler(inputCols=['length', 'tf-idf_features'], outputCol='features')

## The Model
- Create a <b>NaiveBayes</b> classifier with the default parameters.

In [ ]:
nb = NaiveBayes(featuresCol='features', labelCol='label',  predictionCol='prediction')

## Pipeline
### Create a pipeline model contains all the steps starting from the Tokenizer to the NaiveBays classifier.

In [ ]:
myStages = [tokenizer, Remover, Count_Vectorizer, idf, stringIndexer, vecAssembler, nb]
pipeline = Pipeline(stages=myStages)

### Split your data to trian and test data with ratios 0.7 and 0.3 respectively.

In [ ]:
trainDF, testDF = df.randomSplit([0.7,0.3],seed=42)

In [ ]:
trainDF.count()

3981

In [ ]:
testDF.count()

1593

### Fit your Pipeline model to the training data

In [ ]:
pipelineModel = pipeline.fit(trainDF)

### Perform predictions on tests dataframe

In [ ]:
predDF = pipelineModel.transform(testDF)

### Print the schema of the prediction dataframe

In [ ]:
predDF.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = false)
 |-- Tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Cleaned_Tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- CountVectorizerOutput: vector (nullable = true)
 |-- tf-idf_features: vector (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



## Model Evaluation
- Use <b>MulticlassClassificationEvaluator</b> to calculate the <b>f1_score</b>.

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
f1_score = MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='label', metricName = 'f1')

In [ ]:
print("f1_score is:", f1_score.evaluate(predDF))

f1_score is: 0.9727502290227267


# GOOD LUCK
<b><font color='GREEN'>AI-PRO Spark Team ITI</font></b>

![image-3.png](attachment:image-3.png)